In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

In [3]:
'''Function to determine full count ratios WITH summing (S)'''
def FullSum(x, F, Energies, et, ep, Q, alpha, debug, show):        
    
    '''Normalizing X and F'''
    # Normalize branching ratios
    for i in range(0,len(x)):
        xsum = x[i].sum()
        if np.equal(xsum,0)==True:
            for j in range(0,len(x)):
                x[i,j] = x[i,j]
                j=j+1
        else:
            if np.not_equal(xsum,1)==True:
                for j in range(0,len(x)):
                    x[i,j] = x[i,j]/float(xsum)   #float(xsum)
                    j=j+1
        i=i+1  
    
    # Normalize feeding fractions
    fsum=F.sum()
    # print(fsum)
    if F.sum()>1:
        for i in range(0,len(F)):
            F[i] = F[i]/float(fsum)
            
    '''Probabilities'''
        # probability of branch decay occuring (accounting for loss through conversion)
    c = x/(1+alpha) 
        # probability of peak detected at expected photopeak (decay * peak efficiency)
    a = c*ep/Q
        # probability of peak detected anywhere in spectrum (decay * total efficiency)
    e = c*et/Q
        # probability of no gamma ray detection (used for summing out)
    b = x-e

    E = np.identity(w)
    
    '''FULL SUMMING'''
    '''These while loops currently compare float values to zero without a defined tolerance for equality.
    I can set the absolute tolerance using np.isclose() or math.isclose(), but I need to decide on a threshold value.'''

        # summing in 
    aa = a.dot(a)
    asum = aa.sum
    asum = aa.sum(dtype=float)
    while (asum > 0):
        a = a + aa
        aa = np.dot(aa,a)
        asum = aa.sum(dtype=float)
    A = Q*a

        # summing out
    bb = b.dot(b)
    bsum = bb.sum(dtype=float)
    while (bsum > 0):
        b = b + bb
        bb = np.dot(bb,b)
        bsum = bb.sum(dtype=int)
    B = E + b #kth power

    '''Calculating full measured peaks (S)'''

    # EQUATION 6

    n = F.dot(B)
    N = E*n


    m = B[0:w,0]
    M = E*m
    
    S = R*N.dot(A).dot(M)
    
    if debug==True:
        print("Energy levels:",'\n',Energies)
        print("Total efficiency:",'\n',et)
        print('Tline', Tline)
        print("Peak efficiencies:",'\n',ep) 
        print("Conversion coefficients:",'\n',alpha)
        print("alpha caps", astart, aend)
        print("Source disintegration rate:", '\n', R)
        print("Number of detectors:", '\n', Q)
        print("A", '\n', A)
        print("B", '\n', B)
    
    if show==True:
        print("Branching fractions:",'\n',x)
        print("Feeding fractions:",'\n',F)
        print("Summed peaks", '\n', S)
    
    return(S)

In [4]:
'''Function to determine count ratios WITHOUT summing (So)'''
def NoSum(x, F, Energies, et, ep, Q, alpha, debug, show):        
    
    '''Normalizing X and F'''
    # Normalize branching ratios
    for i in range(0,len(x)):
        xsum = x[i].sum()
        if np.equal(xsum,0)==True:
            for j in range(0,len(x)):
                x[i,j] = x[i,j]
                j=j+1
        else:
            if np.not_equal(xsum,1)==True:
                for j in range(0,len(x)):
                    x[i,j] = x[i,j]/float(xsum)   #float(xsum)
                    j=j+1
        i=i+1  
    
    # Normalize feeding fractions
    fsum=F.sum()
    # print(fsum)
    if F.sum()>1:
        for i in range(0,len(F)):
            F[i] = F[i]/float(fsum)
            
    '''Probabilities'''
        # probability of branch decay occuring (accounting for loss through conversion)
    c = x/(1+alpha) 
        # probability of peak detected at expected photopeak (decay * peak efficiency)
    a = c*ep/Q
        # probability of peak detected anywhere in spectrum (decay * total efficiency)
    e = c*et/Q
        # probability of no gamma ray detection (used for summing out)
    b = x-e

    E = np.identity(w)

    '''FULL SUMMING'''
    '''These while loops currently compare float values to zero without a defined tolerance for equality.
    I can set the absolute tolerance using np.isclose() or math.isclose(), but I need to decide on a threshold value.'''

        # summing in 
    aa = a.dot(a)
    asum = aa.sum
    asum = aa.sum(dtype=float)
    while (asum > 0):
        a = a + aa
        aa = np.dot(aa,a)
        asum = aa.sum(dtype=float)
    A = Q*a

        # summing out
    bb = b.dot(b)
    bsum = bb.sum(dtype=float)
    while (bsum > 0):
        b = b + bb
        bb = np.dot(bb,b)
        bsum = bb.sum(dtype=int)
    B = E + b #kth power
    
    '''NO SUMMING'''
    Ao = a
    Bo = E+x

    no = F.dot(Bo)
    No = E*no

    mo = Bo[0:w,0]
    Mo = E

    So = R*No.dot(Ao).dot(Mo)
    
    if debug==True:
        print("Energy levels:",'\n',Energies)
        print("Feeding fractions:",'\n',F)
        print("Branching fractions:",'\n',x)
        print("Total efficiency:",'\n',et)
        print('Tline', Tline)
        print("Peak efficiencies:",'\n',ep) 
        print("Conversion coefficients:",'\n',alpha)
        print("alpha caps", astart, aend)
        print("Source disintegration rate:", '\n', R)
        print("Number of detectors:", '\n', Q)
        print("Ao", '\n', Ao)
        print("Bo",'\n', Bo)
        print("(F . B_o)", '\n', no)
        print("No",'\n', No)
        print("Mo",'\n', Mo)
    
    if show==True:
        print("Peaks without summing:",'\n',So)
    
    return(So)

In [5]:
'''READ IN SOURCE FILE (Forwards)'''
if __name__ == '__main__':
    '''INCLUDE TEXT FILE'''
    Data = np.genfromtxt("Co-60", comments="#", delimiter=",", filling_values='0')
    Efficiencies = np.genfromtxt("efficiency.txt", comments="#", delimiter=",", filling_values='0')

    # Order of matrix (number of energy levels, including the ground state)
    w = len(Data[0])
    '''Defining energy drops (keV)
    This section reads in energy levels and converts them to a matrix containing the energies of gamma rays emitted from all possible energy drops.
    ''' 
    # Creates empty matrix of appropriate dimensions, and reads in data from first line of text file
    Energies = np.zeros((w,w))
    length_end = w+1
    # print(length_end)
    for i in range(0,w):
    # creates empty vector
        v=0
        for j in range(0,len(Data[0])):
    #         Defines energy drops
            k = Data[0,i]-Data[0,j]
            j=j+1
            v=np.append(v,k)
    #     Ensures that matrix only contains one copy of each value
        for d in range(0,len(v)):
            if v[d] < 0:
                v[d]=0
    #     Adds each vector to empty matrix
        Energies[i]=np.array(v[1:length_end])
    # Final energy levels (keV)

    '''Feeding Fractions'''
    # Create vector from file
    F = np.array([Data[1]])

    '''Branching Fractions (NORMALIZE)'''
    # Create matrix from file
    x = np.array(Data[2:2+w])  
    
    '''Total Efficiency'''
    Tline = 2+w
    # If debug=True, print Tline
    T = Data[Tline]
    et = T.sum() #for total array

    '''Peak Efficiencies'''
    # Round energy levels to nearest integer
    ep = np.rint(Energies)
    # print(P)
    for i in range(0,len(ep[0])):
        for j in range(0,len(ep[0])):
            if np.not_equal(ep[i,j],0)==True:
                d=ep[i,j]
    #             Change D type to integer and value to reflect line number in TRIUMF data 
                D=d.astype(int)-9
    #             Change array value to peak efficiency for specified energy
                ep[i,j] = Efficiencies[D,1]

    '''Conversion Coefficients'''
    astart = Tline+1
    aend = astart + w
    # if debug=True, print astart and aend ("alpha caps")
    alpha = np.array(Data[astart:aend])
    ###alpha = np.ones((4,4))*0 #no conversion coefficients for toy system

    '''Source disintegration rate'''
    rline = aend
    r = np.array(Data[rline])
    R = r.sum()

    '''Number of detectors'''
    nline = rline + 1
    q = (Data[nline])
    Q=1

In [6]:
'''USE FUNCTIONS'''    
S=FullSum(x, F, Energies, et, ep, Q, alpha, debug=False, show=True)
So=NoSum(x, F, Energies, et, ep, Q, alpha, debug=False, show=True)

Branching fractions: 
 [[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.36363636e-01   8.63636364e-01   0.00000000e+00   0.00000000e+00]
 [  2.00285403e-08   9.99924873e-01   7.51070260e-05   0.00000000e+00]]
Feeding fractions: 
 [[  0.00000000e+00   1.19997600e-03   1.99996000e-05   9.98780024e-01]]
Summed peaks 
 [[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  6.45859840e+03   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.54754831e-01   4.84115363e-01   0.00000000e+00   0.00000000e+00]
 [  1.24997727e+03   6.97251854e+03   7.59224574e-01   0.00000000e+00]]
Peaks without summing: 
 [[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.07553078e+04   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  2.26185088e-01   1.17928118e+00   0.00000000e+00   0.00000000e+00]
 [  1.24997727e+03   1.16208642e+04   1.93321072e+00   0.0000

In [7]:
'''READ IN PEAK FILES'''
'''Inputs: Measured peaks (S), peak efficiencies (ep), total efficiency (et)'''
PeakData = np.genfromtxt("Co-60 PEAKS.txt", comments="#", delimiter=",", filling_values='0')
Efficiencies = np.genfromtxt("efficiency.txt", comments="#", delimiter=",", filling_values='0')

def PeaktoSource(PeakData, Efficiencies, debug, show):
    # Order of matrices (number of energy levels, including the ground state)
    w = len(PeakData[0])

    '''PROVIDED VALUES'''
    '''Source disintegration rate'''
    rline = 0
    r = np.array(PeakData[rline])
    R = r.sum()
    # R = 100

    '''Total Efficiency'''
    Tline = rline+1
    # print('Tline', Tline)
    T = PeakData[Tline]
    et = T.sum() #for total array
    # et = 1

    '''Conversion Coefficients'''
    astart = Tline+1
    aend = astart + w
    # print("Acaps", astart, aend)
    alpha = np.array(PeakData[astart:aend])
    ###alpha = np.ones((4,4))*0 #no conversion coefficients for toy system

    '''Number of detectors'''
    nline = aend
    q = (PeakData[nline])
    Q = q.sum()

    '''Energy Levels (keV)'''
    # Creates empty matrix of appropriate dimensions, and reads in data from line of text file
    Energies = np.zeros((w,w))
    length_end = w+1
    eline = nline+1
    # print(length_end)
    for i in range(0,w):
    # creates empty vector
        v=0
        for j in range(0,w):
    #         Defines energy drops
            k = PeakData[eline,i]-PeakData[eline,j]
            j=j+1
            v=np.append(v,k)
    #     Ensures that matrix only contains one copy of each value
        for d in range(0,len(v)):
            if v[d] < 0:
                v[d]=0
    #     Adds each vector to empty matrix
        Energies[i]=np.array(v[1:length_end])
    # Final energy levels (keV)

    '''Measured Peaks'''
    MPstart = eline+1
    MPend = MPstart + w
    # print("MPcaps", MPstart, MPend)
    S = np.array(PeakData[MPstart:MPend])

    '''Peak Efficiencies'''
    # Round energy levels to nearest integer
    ep = np.rint(Energies)
    # print(P)
    for i in range(0,len(ep[0])):
        for j in range(0,len(ep[0])):
            if np.not_equal(ep[i,j],0)==True:
                d=ep[i,j]
    #             Change D type to integer and value to reflect line number in TRIUMF data 
                D=d.astype(int)-9
    #             Change array value to peak efficiency for specified energy
                ep[i,j] = Efficiencies[D,1]
    
    if debug==True:
        print('Number of Detectors', '\n', Q)
        print("Conversion coefficients:",'\n',alpha)
        print("Source disintegration rate:", '\n', R)
        print("Total efficiency:",'\n',et)
    
    if show==True:
        print("Energy levels:",'\n', Energies)
        print("Peak efficiencies:",'\n',ep)
        print("Measured Peaks:",'\n',S)        
    
    return(S)

In [8]:
Sm=PeaktoSource(PeakData, Efficiencies, debug=False, show=True)

Energy levels: 
 [[    0.        0.        0.        0.   ]
 [ 1332.508     0.        0.        0.   ]
 [ 2158.61    826.102     0.        0.   ]
 [ 2505.748  1173.24    347.138     0.   ]]
Peak efficiencies: 
 [[ 0.          0.          0.          0.        ]
 [ 0.10756144  0.          0.          0.        ]
 [ 0.07667133  0.14371242  0.          0.        ]
 [ 0.0676444   0.11635693  0.25770852  0.        ]]
Measured Peaks: 
 [[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.06888560e+04   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.00821892e-01   1.16612795e+00   0.00000000e+00   0.00000000e+00]
 [  1.95306310e+01   1.15479991e+04   1.91075843e+00   0.00000000e+00]]


In [9]:
'''Guess branching ratios (Xg)'''
def findX(w, S, ep, debug, show):
    Xg = np.zeros((w,w))
    for i in range(0,len(ep[0])):
        for j in range(0,len(ep[0])):
            if np.not_equal(S[j,i],0)==True:
                Xg[j,i]=S[j,i]/ep[j,i]
# Attempt to remove summing out:
#         for j in range(1,len(ep[0])):
#             if np.not_equal(S[j,i],0)==True:
#                 Xg[j,i]=S[j,i]/ep[j,i]
#         for j in range(0,0):
#             if np.not_equal(S[0,i],0)==True:
#                 Xg[0,i]=(S[0,i]/ep[0,i])-sum(S[0:len(S[0]),i]/ep[0:len(ep[0]),i])
                
    if debug==True:
        print("Peaks",'\n', S)
        print("peak efficiencies",'\n', ep)
        
    if show==True:
        print("Branching ratio guess:",'\n', Xg)
    
    return(Xg)

'''Guess feeding fractions (Fg)'''
def findF(w, S, ep, debug, show):
    Xg = np.zeros((w,w))
    for i in range(0,len(ep[0])):
        for j in range(0,len(ep[0])):
            if np.not_equal(S[i,j],0)==True:
                Xg[i,j]=S[i,j]/ep[i,j]
     
    Fg = np.zeros(w)
    for i in range(0,len(Xg[0])):
        for j in range(0,len(Xg[0])):
            fg = sum(Xg[i,0:len(Xg[0])])-sum(Xg[0:len(Xg[0]),i])
        Fg[i]=fg
        for d in range(0,len(Fg)):
            if Fg[d] < 0:
                Fg[d]=0
            
    if debug==True:
        print("Peaks",'\n', S)
        print("peak efficiencies",'\n', ep)
        print("Branching ratio guess:",'\n', Xg)
        
    if show==True:
        print("Feeding ratio guess:",'\n', Fg)
    
    return(Fg)

In [10]:
Xg=findX(w, S, ep, debug=False, show=True)
Fg=findF(w, S, ep, debug=False, show=True)

Branching ratio guess: 
 [[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  6.00456656e+04   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  2.01841850e+00   3.36863981e+00   0.00000000e+00   0.00000000e+00]
 [  1.84786501e+04   5.99235366e+04   2.94605924e+00   0.00000000e+00]]
Feeding ratio guess: 
 [  0.00000000e+00   1.18760333e+02   2.44099907e+00   7.84051327e+04]


In [11]:
Sg=FullSum(Xg, Fg, Energies, et, ep, Q, alpha, debug=False, show=True)
print("Actual Summed Peaks",'\n', S)
Sdiff=Sg-S
print("Difference", '\n', Sdiff)

Branching fractions: 
 [[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  3.74679163e-01   6.25320837e-01   0.00000000e+00   0.00000000e+00]
 [  2.35681637e-01   7.64280788e-01   3.75748262e-05   0.00000000e+00]]
Feeding fractions: 
 [  0.00000000e+00   1.51236314e-03   3.10851015e-05   9.98456552e-01]
Summed peaks 
 [[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  4.94129918e+03   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  2.05769692e-01   2.88984359e-01   0.00000000e+00   0.00000000e+00]
 [  2.54689155e+03   5.32757846e+03   4.35003877e-01   0.00000000e+00]]
Actual Summed Peaks 
 [[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  6.45859840e+03   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.54754831e-01   4.84115363e-01   0.00000000e+00   0.00000000e+00]
 [  1.24997727e+03   6.97251854e+03   7.59224574e-01   0.00000000e

In [25]:
'''Run loop to keep guessing new x, F'''
# while np.not_equal(Sdiff.any(),0)==True:
Xg=findX(w, S, ep, debug=False, show=False)
for x in range(0, 10):
    Fg=findF(w, Sg, ep, debug=False, show=False)
    Sg=FullSum(Xg, Fg, Energies, et, ep, Q, alpha, debug=False, show=False)
    Sdiff=Sg-S
#     if np.not_equal(Sdiff.all(),0)==False:
#         print("Done!")
    for i in range(0,len(Xg[0])):
        for j in range(0,len(Xg[0])):
            if (Sdiff[i,j]>0):
                Xg[i,j]=Xg[i,j]-Xg[i,j]*(Sg[i,j]/S[i,j])
            if (Sdiff[i,j]<0):
                Xg[i,j]=Xg[i,j]+Xg[i,j]*(Sg[i,j]/S[i,j])   
#     return(Sg)
print("Final Guess", '\n', Sg)
print("Goal" '\n', S)
print("New Difference", '\n', Sdiff)

Final Guess 
 [[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.07561442e+04   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  5.69189446e-08   7.99371298e-08   0.00000000e+00   0.00000000e+00]
 [  1.37182415e-07   2.86957675e-07   2.34304873e-11   0.00000000e+00]]
Goal
 [[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  6.45859840e+03   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.54754831e-01   4.84115363e-01   0.00000000e+00   0.00000000e+00]
 [  1.24997727e+03   6.97251854e+03   7.59224574e-01   0.00000000e+00]]
New Difference 
 [[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  4.29754584e+03   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [ -1.54754774e-01  -4.84115283e-01   0.00000000e+00   0.00000000e+00]
 [ -1.24997727e+03  -6.97251854e+03  -7.59224574e-01   0.00000000e+00]]
